In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import os
from sklearn.utils import class_weight
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, LearningRateScheduler
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.models import Sequential
from sklearn.metrics import classification_report
import seaborn as sns
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [ ]:
# Define common parameters
Img_size = (224, 224)
Batch_Size = 32
Train_Data_Dir = "C:/Users/OWNER/Downloads/SDS-CP028-smart-leaf_Project/SDS-CP028-smart-leaf/submissions/team-members/Samsudeen/Data/Train_Val_Test_Data/train"
Val_Data_Dir = "C:/Users/OWNER/Downloads/SDS-CP028-smart-leaf_Project/SDS-CP028-smart-leaf/submissions/team-members/Samsudeen/Data/Train_Val_Test_Data/val"
Test_Data_Dir = "C:/Users/OWNER/Downloads/SDS-CP028-smart-leaf_Project/SDS-CP028-smart-leaf/submissions/team-members/Samsudeen/Data/Train_Val_Test_Data/test"

# Data Augmentation for Training

In [ ]:

train_datagen = ImageDataGenerator(
    rescale=1./255,                  # Normalize pixels
    rotation_range=20,              # Random rotation
    width_shift_range=0.1,          # Horizontal shift
    height_shift_range=0.1,         # Vertical shift
    shear_range=0.1,                # Shearing
    zoom_range=0.1,                 # Random zoom
    horizontal_flip=True,           # Random horizontal flip
    vertical_flip = True,            # Random Vertical flip
    brightness_range =[0.8, 1.2],  # Random Brightness
    fill_mode='nearest'             # Filling strategy
)


In [ ]:
# Validation/Test: Only Rescaling (NO Augmentation)
test_val_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
# Load datasets
train_generator = train_datagen.flow_from_directory(Train_Data_Dir,
    target_size=Img_size,
    batch_size=Batch_Size,
    class_mode='categorical',
    shuffle=True
)

val_generator = test_val_datagen.flow_from_directory(Val_Data_Dir,
    target_size=Img_size,
    batch_size=Batch_Size,
    class_mode='categorical',
    shuffle=False
)

test_generator = test_val_datagen.flow_from_directory(Test_Data_Dir,
    target_size=Img_size,
    batch_size=Batch_Size,
    class_mode='categorical',
    shuffle=False
)

In [ ]:
class_indices = train_generator.class_indices
print(class_indices)

# Calculate class weights

In [ ]:

class_weights = compute_class_weight(
    "balanced", 
    classes = np.unique(train_generator.classes),
    y =train_generator.classes
)
class_weight_dict = dict(zip(np.unique(train_generator.classes), class_weights))
print(f"Class weights: {class_weight_dict}")

# Define the CNN Model

In [ ]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(2, 2),
    
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    
    Flatten(),
    # Change to 256
    Dense(512, activation='relu'),
    Dropout(0.3),
    
    Dense(14, activation='softmax')  # 14 classes for classification
])

## Compile the model

In [ ]:
model.compile(
    optimizer=Adam(),
    loss='categorical_crossentropy',  # For multi-class classification
    metrics=['accuracy']
)

# Define callbacks: EarlyStopping and ModelCheckpoint

In [ ]:
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
checkpoint = ModelCheckpoint('best_model.h5', save_best_only=True, monitor='val_loss', mode='min')

## Train the model

In [ ]:

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10,  # Adjust the number of epochs as needed
    validation_data=val_generator,
    validation_steps=val_generator.samples // val_generator.batch_size,
    class_weight = class_weight_dict,  # Apply class weights to handle class imbalance
    callbacks=[early_stop, checkpoint]
)

# Plotting the training and validation loss

In [ ]:

plt.figure(figsize=(12, 6))

# Plot training loss
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Plot training accuracy

In [ ]:
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# Display the plots
plt.tight_layout()
plt.show()

In [ ]:
# Save the final model
model.save('final_model.h5')

print("Model training complete and saved!")

# Hyperparameter Tuning

In [ ]:
!pip install keras-tuner

In [ ]:
import keras_tuner as kt

In [ ]:
# Define the model-building function
def build_model(hp):
    model = Sequential()

    # Hyperparameter tuning for the number of filters and kernel size in Conv2D layers
    model.add(layers.Conv2D(filters=hp.Int('conv_1_filters', min_value=32, max_value=128, step=32),
                            kernel_size=hp.Choice('conv_1_kernel', values=[3, 5]),
                            activation='relu',
                            input_shape=(224, 224, 3)))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    
    # Add second Conv2D layer
    model.add(layers.Conv2D(filters=hp.Int('conv_2_filters', min_value=32, max_value=128, step=32),
                            kernel_size=hp.Choice('conv_2_kernel', values=[3, 5]),
                            activation='relu'))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))

    # Flatten layer
    model.add(layers.Flatten())

    # Hyperparameter tuning for the number of units in Dense layer
    model.add(layers.Dense(units=hp.Int('dense_units', min_value=64, max_value=512, step=64), activation='relu'))
    
    # Dropout layer
    model.add(layers.Dropout(rate=hp.Float('dropout_rate', min_value=0.2, max_value=0.5, step=0.1)))
    
    # Output layer
    model.add(layers.Dense(14, activation='softmax'))  # 14 classes for classification

    # Hyperparameter tuning for the learning rate of Adam optimizer
    model.compile(optimizer=Adam(learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [ ]:
from keras_tuner import Hyperband

# Set up Hyperband search
tuner = Hyperband(
    build_model,
    objective='val_accuracy',  # Optimize validation accuracy
    max_epochs=10,  # Max epochs per trial
    factor=3,  # Factor for Hyperband's exploration
    directory='my_dir',  # Directory to store the results
    project_name='hyperparameter_tuning'
)

# Perform hyperparameter search
tuner.search(train_generator, validation_data=val_generator, epochs=10)


In [ ]:
# Get the best hyperparameters
best_hps = tuner.oracle.get_best_trials(num_trials=1)[0].hyperparameters

# Print the best hyperparameters
print(f"Best hyperparameters: {best_hps.values}")


In [ ]:
# Build the model with the best hyperparameters
model = tuner.hypermodel.build(best_hps)

# Train the model
history = model.fit(train_generator, validation_data=val_generator, epochs=10)


In [ ]:
#model.save('Optimised_model.h5')
#model.save('my_model.h5') or keras.saving.save_model(model, 'my_model.h5')
model.save('Optimise_CNN.keras') #or keras.saving.save_model(model, 'Optimise_CNN.keras')

In [ ]:
pip install keras

In [ ]:
import keras

In [ ]:
# Load the best model saved during training
#best_model = tf.keras.models.load_model('Optimised_model.h5')
loaded_model = keras.models.load_model('Optimise_CNN.keras')

In [ ]:
# Get the class indices mapping
class_indices = test_generator.class_indices
inverted_class_indices = {v: k for k, v in class_indices.items()}

print("Attempting to display an image with its name and model prediction...")

try:
    # 1. Get a batch of data and filenames from the test_generator
    sample_data = next(iter(test_generator))
    filenames = test_generator.filenames[test_generator.batch_index * test_generator.batch_size :
                                         (test_generator.batch_index + 1) * test_generator.batch_size]

    # 2. Extract images
    if isinstance(sample_data, tuple) and len(sample_data) == 2:
        input_images, true_labels = sample_data
    else:
        input_images = sample_data
        true_labels = None

    if input_images.size > 0:
        # 3. Select the first image in the batch
        image_to_display_original = input_images[0]
        filename = filenames[0]

        # 4. Preprocess for display
        image_to_display = (image_to_display_original * 255).astype(np.uint8)
        if image_to_display.shape[-1] == 1 and image_to_display.ndim == 3:
            image_to_display = image_to_display.squeeze(axis=-1)

        # 5. Make a prediction
        # Expand dimensions to create a batch of size 1
        img_expanded = np.expand_dims(image_to_display_original, axis=0)
        predictions = loaded_model.predict(img_expanded)
        predicted_class_index = np.argmax(predictions[0])
        predicted_class_name = inverted_class_indices.get(predicted_class_index, "Unknown")

        # Get the true label name if available
        true_class_name = "N/A"
        if true_labels is not None:
            true_class_index = np.argmax(true_labels[0])
            true_class_name = inverted_class_indices.get(true_class_index, "Unknown")

        # 6. Display the image with filename, true label, and prediction
        plt.figure(figsize=(6, 6))
        plt.imshow(image_to_display, cmap='gray' if image_to_display.ndim == 2 else None)
        title = f"Filename: {filename}\nTrue Label: {true_class_name}\nPrediction: {predicted_class_name}"
        plt.title(title)
        plt.axis("off")
        plt.show()

        # Reset the generator to the beginning for the evaluation loop
        test_generator.reset()

    else:
        print("The test_generator yielded an empty batch, cannot display an image.")

except StopIteration:
    print("The test_generator is exhausted or empty. Cannot fetch a sample image.")
    print("You might need to reset your generator if it has been fully iterated through (e.g., test_generator.reset()).")
except Exception as e:
    print(f"An error occurred while trying to display the image: {e}")
    print("Please ensure your test_generator yields data in the expected format (e.g., NumPy arrays).")
    print("Also, double-check image dimensions and all preprocessing steps for plt.imshow(), especially denormalization.")

print("\nProceeding with model evaluation on the test set...")

# Evaluate the model on the test set
if 'loaded_model' in locals():
    test_loss, test_acc = loaded_model.evaluate(test_generator, steps=test_generator.samples // test_generator.batch_size)
    print(f"Test Loss: {test_loss}")
    print(f"Test Accuracy: {test_acc}")
else:
    print("Warning: 'loaded_model' not found, skipping evaluation.")

# Classification report

In [ ]:
# Get true labels
y_true = test_generator.classes  # class indices from the generator

# Get class names
class_names = list(test_generator.class_indices.keys())

# Get predictions
y_pred_probs = best_model.predict(test_generator, steps=test_generator.samples // test_generator.batch_size + 1)
y_pred = np.argmax(y_pred_probs, axis=1)

# Print classification report
print(classification_report(y_true, y_pred, target_names=class_names))


# Confusion Matrix

In [ ]:
# 1. True labels (from the generator)
y_true = test_generator.classes

# 2. Predicted labels
y_pred_probs = best_model.predict(test_generator, steps=test_generator.samples // test_generator.batch_size + 1)
y_pred = np.argmax(y_pred_probs, axis=1)

# 3. Class labels
class_names = list(test_generator.class_indices.keys())

# 4. Confusion matrix
cm = confusion_matrix(y_true, y_pred)

# 5. Plot the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.tight_layout()
plt.show()
